# Working with Tezos Sandbox
Please complete [Installation of tezos from sources](macos-tezos.ipynb) before.

Tezos in sandbox is temporary. After terminating the node all data will be erased. This is usable for quick development and testing. During use, make sure you are saving your work as it might be erased afterwards.


Execute tezos server in separate console (Jupyter don't support running multiple kernels in parallel natively).

In [3]:
#cd ~/tezos-dev/tezos
#./src/bin_node/tezos-sandboxed-node.sh 1 --connections 1 &

[1] 9493


## 1. Initialization of the tezos client
Script will initialize variables. **The variables are present only in current shell. When opening other shell it's required to rerun the command.**

In [6]:
cd ~/tezos-dev/tezos
eval `./src/bin_client/tezos-init-sandboxed-client.sh 1`

  
   The node you are connecting to claims to be running in a
                    Tezos TEST SANDBOX.
      Do NOT use your fundraiser keys on this network.
  You should not see this message if you are not a developer.

## Tezos address added: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx
  
   The node you are connecting to claims to be running in a
                    Tezos TEST SANDBOX.
      Do NOT use your fundraiser keys on this network.
  You should not see this message if you are not a developer.

## Tezos address added: tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN
  
   The node you are connecting to claims to be running in a
                    Tezos TEST SANDBOX.
      Do NOT use your fundraiser keys on this network.
  You should not see this message if you are not a developer.

## Tezos address added: tz1faswCTDciRzE4oJ9jn2Vm2dvjeyA9fUzU
  
   The node you are connecting to claims to be running in a
                    Tezos TEST SANDBOX.
      Do NOT use your fundraiser keys on this netwo

## 2. Initiate blockchain - creation of genesis block

In [7]:
tezos-activate-alpha 2>/dev/null
tezos-client bake for bootstrap1

Injected BMDLXJtBAYco
  
   The node you are connecting to claims to be running in a
                    Tezos TEST SANDBOX.
      Do NOT use your fundraiser keys on this network.
  You should not see this message if you are not a developer.

Injected block BKombhrWwLUb


## 3. List available accounts

By running this command, client will show you list of aliases for registered accounts with sandbox tezos.

In [8]:
tezos-client list known addresses 2>/dev/null

activator: tz1TGu6TN5GSez2ndXXeDX6LgUDvLzPLqgYV (unencrypted sk known)
bootstrap5: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv (unencrypted sk known)
bootstrap4: tz1b7tUupMgCNw2cCLpKTkSD1NZzB5TkP2sv (unencrypted sk known)
bootstrap3: tz1faswCTDciRzE4oJ9jn2Vm2dvjeyA9fUzU (unencrypted sk known)
bootstrap2: tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN (unencrypted sk known)
bootstrap1: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx (unencrypted sk known)


Referencing alias or has is equivalent when using tezos-client. Following commands can be used to retrieve balance on accounts.

In [9]:
tezos-client get balance for tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv 2>/dev/null #bootstrap1

4000000 ꜩ


In [10]:
tezos-client get balance for bootstrap1 2>/dev/null

3999488 ꜩ


## 4. Get account keys
Following command will get account keys. This keys are later referenced when deploying contracts.

In [11]:
tezos-client show address bootstrap1 --show-secret 2>/dev/null


Hash: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx
Public Key: edpkuBknW28nW72KG6RoHtYW7p12T6GKc7nAbwYX5m8Wd9sDVC9yav
Secret Key: unencrypted:edsk3gUfUPyBSfrS9CCgmCiQsTCHGkviBDusMxDJstFtojtc1zcpsh


## 5. Deployment of testing liquidity contract


Liquidity is a language heavily inspired by the OCaml language. Before starting working with Liquidity it is advised 
to check OCaml tutorial.

Each liquidity file consist of three parts
- version declaration
- storage definition
- entry functions 
  - at least method main is required. This method is called when contract is executed.

**add_contract.liq**
```ocaml
[%%version 0.4]
  
type storage = int

let%init storage (init_storage : int) =
  init_storage

let%entry main (parameter : int) storage =
  ( ([] : operation list), storage + parameter )
```

In [12]:
mkdir -p ~/tezos-dev/adding-dapp
cd ~/tezos-dev/adding-dapp

/bin/cat <<EOM >add.liq
[%%version 0.4]
  
type storage = int

let%init storage (init_storage : int) =
  init_storage

let%entry main (parameter : int) storage =
  ( ([] : operation list), storage + parameter )
  
let%entry subtract (parameter : int) storage =
  ( ([] : operation list), storage - parameter )
EOM



It's required to execute baking operation few times.

In [13]:
TZ_HASH=$(tezos-client show address bootstrap1 --show-secret 2>/dev/null | grep Hash | cut -d" " -f2)
TZ_SECRET=$(tezos-client show address bootstrap1 --show-secret 2>/dev/null | grep Secret | cut -d":" -f3)

liquidity \
  --tezos-node 127.0.0.1:18731 \
  --amount 2tz \
  --fee 0tz \
  --source $TZ_HASH \
  --private-key $TZ_SECRET \
  add.liq \
  --deploy '0' | tee contract_deploy_output.txt
  
export CONTRACT_HASH=$(grep "New contract" contract_deploy_output.txt | cut -d" " -f3)
export OPERATION_HASH=$(grep "New contract" contract_deploy_output.txt | cut -d" " -f7)

Main contract Add
Main contract Add
Evaluated initial storage: 0
New contract KT1T1bGD7JqsUsx2AsBJZydtSt22kbN1wW2i deployed in operation opAzeZ6dsSC5JQ6h47bjtjyxYimbEy7kLUbmgaVjTjrgGFKMj3Y


Bake needs to be called so the contract is included into the blockchain.

In [16]:
tezos-client bake for bootstrap1 2>/dev/null

Injected block BLcR9ipMqZaB


In [14]:
curl -s http://127.0.0.1:18731/chains/main/blocks/head | jq 

{
  "protocol": "PsYLVpVvgbLhAhoqAkMFUo6gudkJ9weNXhUYCiLDzcUpFpkk8Wt",
  "chain_id": "NetXZUqeBjDnWde",
  "hash": "BLMbKrgTZyLopkCnVg354JuSK6usZrS7xhAHov7HFQBTA2ZPK71",
  "header": {
    "level": 3,
    "proto": 1,
    "predecessor": "BMPKQtBvBriZwRDVRNa9ELWeGUWunM3pQ6fnzyLq5eaxFnDFbJJ",
    "timestamp": "2018-12-04T17:58:08Z",
    "validation_pass": 4,
    "operations_hash": "LLoa3iypvUTAxsmLQpLFCMMe7s9jxrn217dTmepHaU7BZzpcVLLf7",
    "fitness": [
      "00",
      "0000000000000003"
    ],
    "context": "CoVyYxxzeHo8caHUHLAkYGzVk3mcaJnsJvx6kGyyHnkJei7WMEyJ",
    "priority": 1,
    "proof_of_work_nonce": "3733b5c5476e3b6a",
    "signature": "sigrSfQqBa1F8dfWPV833E733x55SgcUHKvkqMWnb21Mmm95YgSxvKBceugkDvjzemWRP5Rej67Ax4Gho5JLo98C5WQ8xc2Y"
  },
  "metadata": {
    "protocol": "PsYLVpVvgbLhAhoqAkMFUo6gudkJ9weNXhUYCiLDzcUpFpkk8Wt",
    "next_protocol": "PsYLVpVvgbLhAhoqAkMFUo6gudkJ9weNXhUYCiLDzcUpFpkk8Wt",
    "test_chain_status": {
      "status": "not_running"
    },
    "max_operation

                      "prim": "int"
                    }
                  ]
                },
                {
                  "prim": "code",
                  "args": [
                    [
                      {
                        "prim": "DUP"
                      },
                      {
                        "prim": "DIP",
                        "args": [
                          [
                            {
                              "prim": "CDR",
                              "annots": [
                                "@storage_slash_1"
                              ]
                            }
                          ]
                        ]
                      },
                      {
                        "prim": "CAR",
                        "annots": [
                          "@parameter_slash_2"
                        ]
                      },
                      {
                        "prim": "DUP",
                    

                        "args": [
                          [
                            {
                              "prim": "DROP"
                            },
                            {
                              "prim": "DROP"
                            }
                          ]
                        ]
                      }
                    ]
                  ]
                }
              ],
              "storage": {
                "int": "0"
              }
            },
            "metadata": {
              "balance_updates": [],
              "operation_result": {
                "status": "applied",
                "balance_updates": [
                  {
                    "kind": "contract",
                    "contract": "tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx",
                    "change": "-566000"
                  },
                  {
                    "kind": "contract",
                    "contract": "tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG

After the contract is deployed, it's possible to test it.

In [15]:
liquidity \
  --tezos-node localhost:18732 \
  --amount 5tz \
  --fee 0tz \
  --private-key $TZ_SECRET \
  --source $TZ_HASH \
  add.liq \
  --call $CONTRACT_HASH subtract '2'
  
tezos-client bake for bootstrap1 2>/dev/null

Main contract Add
Successful call to contract KT194pubVP6uqb8cBjpbELjuH2iGayc1K45J in operation oo7HzbHVYGrTbaCQSqo9pYUGz3eQCg49ReQVPdArCbW1UN4avns
Injected block BLZ7sDkqRypi


### TODO
- Forging of contract
- clarification on spendable and delegatable contract

## 4. Deployment of contract in Michelson

In [16]:
liquidity add.liq

Main contract Add
Storage initializer generated in "./add.liq.initializer.tz"
File "./add.liq.tz" generated
If tezos is compiled, you may want to typecheck with:
  tezos-client typecheck script ./add.liq.tz


In [ ]:
tezos-client originate contract first_explosion for bootstrap1 \
             transferring 0 from bootstrap1 \
             running add.liq.tz -init 0

In [ ]:
tezos-client transfer 0 from bootstrap1 to first_explosion -arg 5